# Run simulation with interactive OMC session
This example illustrates how to develop, run, test(, repeat) a Modelica model in a (local) pandas environment, enabling quick iterations to analyze and tune the model before going to Big Data.

See https://openmodelica.org/doc/OpenModelicaUsersGuide/v1.11.0/ompython.html

In [ ]:
import pandas as  pd
import os
import sys

In [ ]:
# Finds the package in the repo instead of the installed one
sys.path.insert(0, '../../../src')

In [ ]:
os.system("which omc")

In [ ]:
import tempfile
from OMPython import ModelicaSystem
import DyMat
from MM import EDrives

## Either build model and run simulation with interactive OMC session...
This section demonstrates how to simulate and of the model using the [OMPython API](https://www.openmodelica.org/doc/OpenModelicaUsersGuide/latest/ompython.html) ***with*** a connection to omc.

In [ ]:
# os.environ['MODELICAPATH'] = os.path.join(sys.prefix, 'lib','omlibrary')  # https://openmodelica.org/doc/OpenModelicaUsersGuide/latest/packagemanager.html
os.environ['OPENMODELICALIBRARY'] = os.path.join(sys.prefix, 'lib','omlibrary')  # https://trac.openmodelica.org/OpenModelica/ticket/6078

In [ ]:
# mod = BouncingBall.instantiatemodel()  # Does NOT connect to omc
modelname = "EDrives.Examples.DCDC.DC_Drive_Continuous"
# modelname = "EDrives.Examples.DCDC.DC_Drive_Switching"
# xmlFilePath = os.path.join(SETUP_DIR, 'MM', 'build', modelname)
mod = ModelicaSystem(
        fileName=os.path.join(os.path.dirname(EDrives.__file__), 'EDrives', 'package.mo'),
        modelName=modelname, lmodel=['Modelica'],
    )

## Or build a model without connecting to OMC
This section demonstrate how to build a model using modelica `*.mos` script files ***without*** connection to omc.

In [ ]:
r = EDrives.buildmodel("EDrives.Examples.DCDC.DC_Drive_Continuous")
r

In [ ]:
mod = EDrives.instantiatemodel("EDrives.Examples.DCDC.DC_Drive_Continuous")

## Working with the model

In [ ]:
sim_options_d = mod.getSimulationOptions()
sim_options_d['stopTime'] = 24.0
mod.setSimulationOptions(sim_options_d)

In [ ]:
mod.getSimulationOptions()

In [ ]:
p = pd.to_numeric(pd.Series(mod.getParameters()), errors='coerce')
p['dcpmData.VaNominal'] 

In [ ]:
dMin = float(mod.getParameters(["dMin"])[0])
dMax = 1.0 - dMin
tauNominal=p['dcpmData.VaNominal'] * p['dcpmData.IaNominal'] / p['dcpmData.wNominal']  # The mode uses ViNominal

In [ ]:
# Create some input timeseries
ts_df1 = pd.DataFrame(columns=[
    'time', 'value'], data=[
    [0.0, 0.5],
    [3.0, 0.5],
    [4.0, dMax],
    [12.0, dMax],
    [13.0, dMin],
    [21.0, dMin],
    [22.0, 0.5],
    [24.0, 0.5],    
])
ts_df1['input'] = 'dutyCycle_series'
ts_df2 = pd.DataFrame(columns=[
    'time', 'value'], data=[
    [0.0, 0.0],
    [6.0, 0.0],
    [7.0, -tauNominal],
    [9.0, -tauNominal],
    [10.0, tauNominal],
    [15.0, tauNominal],
    [16.0, -tauNominal],
    [18.0, -tauNominal],
    [19.0, 0.0],
    [24.0, 0.0],
])
ts_df2['input'] = 'loadTorque_Series'
ts_df = pd.concat([ts_df1, ts_df2], ignore_index=True)
# ts_df['run_key'] = 'r1'
ts_df = ts_df.pivot(index='time', columns='input', values='value').ffill().reset_index()#.drop('input', axis=1)
ts_df['run_key'] = 'r1'
#ts_df.to_csv('test.csv', sep=',', line_terminator=',\n')

In [ ]:
temp_dir = tempfile.gettempdir()

In [ ]:
inputs_required = set(mod.getInputs().keys())
inputs_available = set(ts_df.columns)
if inputs_available.intersection(inputs_required) != inputs_required:
    raise ValueError(
        'Required input series not provided in the DataFrame.\nRequired: {0}. Provided:{1}'.format(
            str(inputs_required), str(inputs_available)
        )
    )
print(str(['time']+list(mod.getInputs().keys())))
pdf = ts_df[['time']+list(mod.getInputs().keys())]
# Write csv file and tell the model class
mod.csvFile = os.path.join(temp_dir, mod.modelName + '_inputs.csv')
pdf.to_csv(mod.csvFile, sep=',', lineterminator=',\n')
mod.inputFlag = True

In [ ]:
%%time
res_vars = ['dcpm.wMechanical', 'constantVoltage.i', 'dcpm.brush.i', 'torque.tau', 'hBridge.ref']
resfilepathname = os.path.join(temp_dir, mod.modelName + '_result.mat')
sim_res = mod.simulate(
        resultfile=resfilepathname,
        simflags=None,
        overrideaux='stopTime=24,variableFilter="'+'|'.join(res_vars)+'"'
    )

In [ ]:
# Either load the result file through DyMat (faster)
def dymat2pandas(dm, block, names) -> pd.DataFrame:
    ts_df = pd.DataFrame(dm.getVarArray(names)).T
    ts_df.columns=['time'] + names
    return ts_df


dm = DyMat.DyMatFile(resfilepathname)
res_df = dymat2pandas(dm, 2, res_vars)
res_df

In [ ]:
%matplotlib inline
#%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(figsize=(5,3.5))
ax.plot(res_df['time'], res_df['dcpm.wMechanical'])
plt.tight_layout()

In [ ]:
%%time
# orload the result file through OMPython
# TODO Loading results (from mat-file) is very slow
data = mod.getSolutions(varList=list(res_vars), resultfile=resfilepathname)
res_df = pd.DataFrame(data).T
res_df.columns = res_vars
res_df

## Testing the build script

In [ ]:
from ModelicaBuildTools import build_script

In [ ]:
modelname = "EDrives.Examples.DCDC.DC_Drive_Continuous"

r = build_script(EDrives, modelname, cwd=os.getcwd())
# modelname = "EDrives.Examples.DCDC.DC_Drive_Switching"
# xmlFilePath = os.path.join(SETUP_DIR, 'MM', 'build', modelname)

# mod = ModelicaSystem(
#         fileName=os.path.join(os.path.dirname(EDrives.__file__), 'EDrives', 'package.mo'),
#         modelName=modelname, lmodel=['Modelica'],
#     )

In [ ]:
r